In [ ]:
using AcausalNets
using QI
using LightGraphs
using LinearAlgebra
import AcausalNets.Inference: infer_naive, infer_naive_debug
import AcausalNets.Systems: multiply_star, multiply_kron
import AcausalNets.Algebra: star

## Setup of variables in the game and the strategy of the player

In [ ]:
var_a = Variable(:a, 3) # the placement of the prize
var_b = Variable(:b, 3) # the initial choice of the door by the player
var_c = Variable(:c, 3) # the door opened by the host
var_d = Variable(:d, 3) # the door opened by the player
var_e = Variable(:e, 2) # whether the player has won (0 = lost, 1 = won)

# rules of the host
roCwAB = Diagonal([
        0,1/2,1/2, #A=0, B=0
        0,0,1, #A=0, B=1
        0,1,0, #A=0, B=2
        0,0,1, #A=1, B=0
        1/2,0,1/2, #A=1, B=1
        1,0,0, #A=1, B=2
        0,1,0, #A=2, B=0
        1,0,0, #A=2, B=1
        1/2,1/2,0 #A=2, B=2
        ]); #

sys_c_ab = DiscreteQuantumSystem([var_a, var_b], [var_c], roCwAB)

# this matrix represents the optimal strategy in classical case 
# - always changing the choice
roDwBC_changing = Diagonal([
        0,0,0, # B=0, C=0 (impossible situation)
        0,0,1, #B=0, C=1 
        0,1,0, #B=0, C=2
        0,0,1, #B=1, C=0
        0,0,0, #B=1, C=1 (impossible situation)
        1,0,0, #B=1, C=2
        0,1,0, #B=2, C=0
        1,0,0, #B=2, C=1
        0,0,0 #B=2, C=2 (impossible situation)
        ])
# this matrix represents the unoptimal strategy in classical case 
# - never changing the choice
# this matrix could actually be simplified to be roDwB 
# since in this case, D == B and C is irrelevant
# but we'll keep the matrices in the same shape in order to later 
# test mixed strategies of the player.
roDwBC_staying = Diagonal([
        0,0,0, # B=0, C=0 (impossible situation)
        1,0,0, #B=0, C=1 
        1,0,0, #B=0, C=2
        0,1,0, #B=1, C=0
        0,0,0, #B=1, C=1 (impossible situation)
        0,1,0, #B=1, C=2
        0,0,1, #B=2, C=0
        0,0,1, #B=2, C=1
        0,0,0 #B=2, C=2 (impossible situation)
        ])

sys_d_bc_changing = DiscreteQuantumSystem([var_b, var_c], [var_d], roDwBC_changing)
sys_d_bc_staying = DiscreteQuantumSystem([var_b, var_c], [var_d], roDwBC_staying)
sys_d_bc = sys_d_bc_changing

roEwAD = Diagonal([
        0,1, # A=0, D=0 (player wins) 
        1,0, #A=0, D=1 
        1,0, #A=0, D=2
        1,0, #A=1, D=0
        0,1, #A=1, D=1 (player wins)
        1,0, #A=1, D=2 
        1,0, #A=2, D=0
        1,0, #A=2, D=1
        0,1 #A=2, D=2 (player wins)
        ])
sys_e_ad = DiscreteQuantumSystem([var_a, var_d], [var_e], roEwAD);

## Different variations of $\rho_{AB}$ and so, the game itself

### Classic example

In [ ]:
roA_classic = Diagonal([1/3, 1/3, 1/3])
sys_a_classic = DiscreteQuantumSystem([var_a], roA_classic)

roB_classic = Diagonal([1/3, 1/3, 1/3])
sys_b_classic = DiscreteQuantumSystem([var_b], roB_classic)

an_classic = AcausalNet()

push!(an_classic, sys_a_classic)
push!(an_classic, sys_b_classic)
push!(an_classic, sys_c_ab)
push!(an_classic, sys_d_bc)
push!(an_classic, sys_e_ad)
show(an_classic)

# variables(an_classic)

### $a$ and $b$ entangled, so that they always have identical values

In [ ]:
roAB_same =1/3*(ket(0,9)+ket(4,9)+ket(8,9))* (bra(0,9)+bra(4,9)+bra(8,9)) 
sys_ab_same = DiscreteQuantumSystem([var_a, var_b], roAB_same)

an_same = AcausalNet()
push!(an_same, sys_ab_same)
push!(an_same, sys_c_ab)
push!(an_same, sys_d_bc)
push!(an_same, sys_e_ad)
show(an_same)

### $a$ and $b$ entangled, so that they always have different values

In [ ]:
roAB_different = 1/6*(ket(1,9)+ket(3,9))*(bra(1,9)+bra(3,9))+ 
1/6*(ket(2,9)+ket(6,9))*(bra(2,9)+bra(6,9))+
1/6*(ket(5,9)+ket(7,9))*(bra(5,9)+bra(7,9))

sys_ab_different = DiscreteQuantumSystem([var_a, var_b], roAB_different)
an_different = AcausalNet()

push!(an_different, sys_ab_different)
push!(an_different, sys_c_ab)
push!(an_different, sys_d_bc)
push!(an_different, sys_e_ad)
show(an_different)

## Inference example

In [ ]:
an = an_same
inferrer = Inferrer(an)
println(variables(inferrer))
show(inferrer)

In [ ]:
a = 0
b = 0
c = 1
d = 2
e = 0

a_obs = Evidence([var_a], ketbra(a,a,3))
b_obs = Evidence([var_b], ketbra(b,b,3))
c_obs = Evidence([var_c], ketbra(c,c,3))
d_obs = Evidence([var_d], ketbra(d,d,3))
e_obs = Evidence([var_e], ketbra(e,e,2))

observations = Evidence{Matrix{Complex{Float64}}}[
#     a_obs,
#     b_obs,
#     c_obs,
#     d_obs,
#     e_obs
    ] 

inferred_system = infer(inferrer, [var_e], observations)

println([v.name for v in variables(inferred_system)])
real(distribution(inferred_system))

## Finding the $\rho_{AB}$ which yields the Nash equilibrium between the player and the host

In [ ]:
function mixed_game_an(
        AB_mix::Float64, 
        DwBC_mix::Float64 = 0.0 # by default, player never changes the door
    )
    
    roAB = roAB_same * AB_mix + roAB_different * (1 - AB_mix)
    roDwBC = roDwBC_changing * DwBC_mix + roDwBC_staying * (1 - DwBC_mix)
    sys_ab = DiscreteQuantumSystem([var_a, var_b], roAB)
    sys_d_bc = DiscreteQuantumSystem([var_b, var_c], [var_d], roDwBC)
    
    result_an = AcausalNet()
    push!(result_an, sys_ab)
    push!(result_an, sys_c_ab)
    push!(result_an, sys_d_bc)
    push!(result_an, sys_e_ad)
    result_an, roAB
end

In [ ]:
function expected_win_rate(an::AcausalNet)
    inferrer = Inferrer(an)
    ev = Evidence{Matrix{Complex{Float64}}}[] # evidence placeholder
    inferred = infer(inferrer, [var_e], ev)
    real(distribution(inferred)) #, debug
end

In [ ]:
lambda = 0.5
mixed_an, AB= mixed_game_an(lambda)
expected_win_rate(mixed_an)